In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 35040455
paper_name = 'pallares_abergel_2022' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [20]:
original_data = pd.read_excel('raw_data/d1mo00267h1.xlsx', sheet_name='Dataset S1', skiprows=1)

In [21]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4291 x 12


In [22]:
original_data.head()

,ORF,Standard name,IC5 - 10 generations,IC5 - 15 generations,IC10 - 10 generations,IC10 - 15 generations,IC20 - 10 generations,IC20 - 15 generations,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,YPL057C,SUR1,-7.299736,-7.091270,-7.681390,-6.699875,-7.241336,-6.613481,NaN,NaN,NaN,NaN
1,YJL004C,SYS1,-1.375073,-4.250898,-1.995468,-2.828876,-1.233006,-1.445272,NaN,NaN,NaN,NaN
2,YNL051W,COG5,-1.842342,-4.923672,-3.464857,-5.325909,-1.816059,-2.720214,NaN,NaN,NaN,NaN
3,YPL056C,LCL1,-4.310673,-6.440544,-6.791194,-6.560099,-6.128555,-5.860379,NaN,NaN,NaN,NaN
4,YNL041C,COG6,-3.585637,-6.844105,-2.866544,-3.740802,-2.070289,-1.935632,NaN,NaN,NaN,NaN


In [23]:
original_data['orf'] = original_data['ORF'].astype(str)

In [24]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [25]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Standard name, IC5 - 10 generations, IC5 - 15 generations, IC10 - 10 generations, IC10 - 15 generations, IC20 - 10 generations, IC20 - 15 generations, Unnamed: 8, Unnamed: 9, Unnamed: 10, Unnamed: 11, orf]
Index: []


In [27]:
original_data.set_index('orf', inplace=True)

In [28]:
original_data = original_data[['IC5 - 10 generations',
                              'IC5 - 15 generations',
                              'IC10 - 10 generations',
                              'IC10 - 15 generations',
                              'IC20 - 10 generations',
                              'IC20 - 15 generations']].copy()

In [29]:
original_data = original_data.groupby(original_data.index).mean()

In [30]:
original_data.shape

(4270, 6)

In [32]:
original_data[original_data.isnull()] = 0

In [33]:
original_data.head()

,IC5 - 10 generations,IC5 - 15 generations,IC10 - 10 generations,IC10 - 15 generations,IC20 - 10 generations,IC20 - 15 generations
orf,,,,,,
YAL001C,0.0,0.0,0.0,0.0,0.0,0.0
YAL002W,0.0,0.0,0.0,0.0,0.0,0.0
YAL004W,0.0,0.0,0.0,0.0,0.0,0.0
YAL005C,0.0,0.0,0.0,0.0,0.0,0.0
YAL007C,0.0,0.0,0.0,0.0,0.0,0.0


# Prepare the final dataset

In [34]:
data = original_data.copy()

In [35]:
dataset_ids = [22212,22213,22214,22215,22216,22217]
datasets = datasets.reindex(index=dataset_ids)

In [36]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [37]:
data.head()

dataset_id,22212,22213,22214,22215,22216,22217
data_type,value,value,value,value,value,value
orf,,,,,,
YAL001C,0.0,0.0,0.0,0.0,0.0,0.0
YAL002W,0.0,0.0,0.0,0.0,0.0,0.0
YAL004W,0.0,0.0,0.0,0.0,0.0,0.0
YAL005C,0.0,0.0,0.0,0.0,0.0,0.0
YAL007C,0.0,0.0,0.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [38]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 21


In [39]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22212,22213,22214,22215,22216,22217
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
1,YAL001C,0.0,0.0,0.0,0.0,0.0,0.0
2,YAL002W,0.0,0.0,0.0,0.0,0.0,0.0
1863,YAL004W,0.0,0.0,0.0,0.0,0.0,0.0
4,YAL005C,0.0,0.0,0.0,0.0,0.0,0.0
5,YAL007C,0.0,0.0,0.0,0.0,0.0,0.0


# Normalize

In [40]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [41]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [42]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,22212,22213,22214,22215,22216,22217,22212,22213,22214,22215,22216,22217
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
1,YAL001C,0.0,0.0,0.0,0.0,0.0,0.0,0.012998,-0.093636,-0.031099,-0.059113,-0.007097,-0.078213
2,YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.012998,-0.093636,-0.031099,-0.059113,-0.007097,-0.078213
1863,YAL004W,0.0,0.0,0.0,0.0,0.0,0.0,0.012998,-0.093636,-0.031099,-0.059113,-0.007097,-0.078213
4,YAL005C,0.0,0.0,0.0,0.0,0.0,0.0,0.012998,-0.093636,-0.031099,-0.059113,-0.007097,-0.078213
5,YAL007C,0.0,0.0,0.0,0.0,0.0,0.0,0.012998,-0.093636,-0.031099,-0.059113,-0.007097,-0.078213


# Print out

In [43]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [44]:
from IO.save_data_to_db3 import *

In [46]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 35040455...
Inserting the new data...


100%|██████████| 6/6 [00:32<00:00,  5.46s/it]

Updating the data_modified_on field...
